# Tensor board
* 그래프 구조 및 변수  저장
    * 네임 스코프 지정
        * 깔끔한 분류
        * `with tf.name_scope('optimizer'):`
    * 이름 표시
        * 변수 및 오프레이터에 name 지정
        * `W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')`
    * 로그 파일 저장
        * `writer = tf.summary.FileWriter('./logs/dnn', tf.get_default_graph())`
* 텐서 보드 실행
    * `tensorboard --logdir=./logs`
* 웹 브라우저로 접속
    * http://localhost:6006

In [1]:
import tensorflow as tf
import numpy as np

data = np.loadtxt('./data/data.csv', delimiter=',',
                  unpack=True, dtype='float32')

x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])
print(x_data, x_data.shape, x_data.dtype)

#########
# 신경망 모델 구성
######
global_step = tf.Variable(0, trainable=False, name='global_step')

X = tf.placeholder(tf.float32, name="phX")
Y = tf.placeholder(tf.float32, name="phY")

with tf.name_scope('layer1'):
    W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.), name='W1')
    L1 = tf.nn.relu(tf.matmul(X, W1))

    tf.summary.histogram("X", X)
    tf.summary.histogram("Weights", W1)

with tf.name_scope('layer2'):
    W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.), name='W2')
    L2 = tf.nn.relu(tf.matmul(L1, W2))

    tf.summary.histogram("Weights", W2)

with tf.name_scope('output'):
    W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.), name='W3')
    model = tf.matmul(L2, W3)

    tf.summary.histogram("Weights", W3)
    tf.summary.histogram("Model", model)

with tf.name_scope('optimizer'):
    cost = tf.reduce_mean(
        tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=model))

    optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
    train_op = optimizer.minimize(cost, global_step=global_step)

    tf.summary.scalar('cost', cost)

#########
# 신경망 모델 학습
######
sess = tf.Session()
saver = tf.train.Saver(tf.global_variables())

ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    print('exist')
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    print("not exit")
    sess.run(tf.global_variables_initializer())

merged = tf.summary.merge_all()
writer = tf.summary.FileWriter('./logs', sess.graph)

for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})

    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

    summary = sess.run(merged, feed_dict={X: x_data, Y: y_data})
    writer.add_summary(summary, global_step=sess.run(global_step))

saver.save(sess, './model/dnn.ckpt', global_step=global_step)

#########
# 결과 확인
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

is_correct = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(is_correct, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

[[0. 0.]
 [1. 0.]
 [1. 1.]
 [0. 0.]
 [0. 0.]
 [0. 1.]] (6, 2) float32
Instructions for updating:
Colocations handled automatically by placer.
not exit
Step: 1,  Cost: 1.713
Step: 2,  Cost: 1.569
Step: 3,  Cost: 1.442
Step: 4,  Cost: 1.330
Step: 5,  Cost: 1.232
Step: 6,  Cost: 1.145
Step: 7,  Cost: 1.067
Step: 8,  Cost: 0.998
Step: 9,  Cost: 0.941
Step: 10,  Cost: 0.895
Step: 11,  Cost: 0.857
Step: 12,  Cost: 0.821
Step: 13,  Cost: 0.784
Step: 14,  Cost: 0.750
Step: 15,  Cost: 0.719
Step: 16,  Cost: 0.692
Step: 17,  Cost: 0.668
Step: 18,  Cost: 0.648
Step: 19,  Cost: 0.631
Step: 20,  Cost: 0.617
Step: 21,  Cost: 0.605
Step: 22,  Cost: 0.594
Step: 23,  Cost: 0.586
Step: 24,  Cost: 0.578
Step: 25,  Cost: 0.572
Step: 26,  Cost: 0.568
Step: 27,  Cost: 0.565
Step: 28,  Cost: 0.562
Step: 29,  Cost: 0.560
Step: 30,  Cost: 0.558
Step: 31,  Cost: 0.557
Step: 32,  Cost: 0.556
Step: 33,  Cost: 0.555
Step: 34,  Cost: 0.555
Step: 35,  Cost: 0.554
Step: 36,  Cost: 0.554
Step: 37,  Cost: 0.553
Step: 3

## 모델 저장
* `saver = tf.train.Saver(tf.global_variables())`
* `saver.save(sess, './model/dnn_final.ckpt')`

In [2]:

saver.save(sess, './model/dnn_final.ckpt')

'./model/dnn_final.ckpt'